In [1]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [652]:
#Library Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import statistics
import math

import gurobipy as gp
from gurobipy import GRB
#Execution Time


start_time = time.time()

---

In [976]:
gurobi_inputs = pd.read_excel('RPL_Output48/48FC_1.xlsx')

#UNIQUE VALUES

unique_stores = gurobi_inputs['Stores'].unique()
unique_SKU = gurobi_inputs['SKU'].unique()
unique_Category = gurobi_inputs['Category'].unique()


In [977]:
new = gurobi_inputs['SKU'].str.split("_", n = 2, expand = True)
new = new[0]+'_' +new[1]

In [978]:
gurobi_inputs['AssortmentCode']= new

In [979]:
unique_Assortment = gurobi_inputs['AssortmentCode'].unique()

In [980]:
#index Reset
gurobi_inputs.reset_index(inplace = True)
del gurobi_inputs['index']


In [981]:
#Define H as the set of stores

H = []

for i in range(0,len(unique_stores)):
    H.append(unique_stores[i])
    
    
#Define J as the set of assortments

J = []
for i in range(0,len(unique_Assortment)):
    J.append(unique_Assortment[i])

    
#Define K as the set of products

K = []

for i in range(0,len(unique_SKU)):
    K.append(unique_SKU[i])
    
#Define L as the set of categories

L = []

for i in range(0,len(unique_Category)):
    L.append(unique_Category[i])



In [982]:
I_initial = {}
DU = {}
DL = {}
invlevel = gurobi_inputs.groupby(['SKU']).sum().reset_index()
IW = {}
W2_max = {}
W2_min = {}
F = {}
a = {}
P = {}
CC = {}
C = {}
e = {}
b = {}
g = {}
n = {} # Number of product k that can be in assortment j

In [983]:
#Initial Stok Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        I_initial[H[j],K[i]] = 0

for i in range(0,len(gurobi_inputs)):
    I_initial[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = gurobi_inputs['Initial_Stock'][i]
    i += 1

In [984]:
CapacityStore = pd.read_excel('StoreCapacityDeneme.xlsx')
CapacityCategory = pd.read_excel('CategoryCapacityDenemeV2.xlsx')

In [985]:
#C Dict
for i in range(0,len(CapacityStore)):
    C[(CapacityStore['Stores'][i])] = CapacityStore['Store Capacity'][i]
    i += 1

In [986]:
#CC Dict
for i in range(0,len(CapacityCategory)):
    CC[(CapacityCategory['Stores'][i],CapacityCategory['Category'][i])] = CapacityCategory['Capacity'][i]
    i += 1
       

In [987]:
#b Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        b[H[j],K[i]] = 0

for i in range(0,len(gurobi_inputs)):
    b[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = 1
    i += 1

In [988]:
#e Dict
for i in range(0,len(L)):
    for j in range(0,len(K)):
        e[L[i],K[j]] = 0

for i in range(0,len(gurobi_inputs)):
    e[(gurobi_inputs['Category'][i],gurobi_inputs['SKU'][i])] = 1

In [989]:
#DU Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        DU[H[j],K[i]] = 0
        
for i in range(0, len(gurobi_inputs)):
    DU[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = gurobi_inputs['Upper'][i]
    i += 1

In [990]:
#DL Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        DL[H[j],K[i]] = 0

for i in range(0, len(gurobi_inputs)):
    DL[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = gurobi_inputs['Lower'][i]
    i += 1

In [991]:
#IW Dict
for i in range(0,len(K)):
    IW[K[i]] = 0

for i in range(0, len(invlevel)):
    IW[(invlevel['SKU'][i])] = invlevel['IW'][i]
    i += 1

In [992]:
#W2_Max Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        W2_max[H[j],K[i]] = 0


for i in range(0,len(gurobi_inputs)):
    W2_max[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = gurobi_inputs['Wmax'][i]
    i += 1

In [993]:
#W2_Min Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        W2_min[H[j],K[i]] = 0

for i in range(0,len(gurobi_inputs)):
    W2_min[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = gurobi_inputs['Wmin'][i]
    i += 1

In [994]:
#F forecast Dict
for i in range(0,len(K)):
    for j in range(0,len(H)):
        F[H[j],K[i]] = 0


for i in range(0,len(gurobi_inputs)):
    F[(gurobi_inputs['Stores'][i], gurobi_inputs['SKU'][i])] = (gurobi_inputs['Lower'][i] + 
    gurobi_inputs['Upper'][i]) / 2
    i += 1

In [995]:
#Holding Cost a_hk
for i in range(0,len(K)):
    for j in range(0,len(H)):
        a[H[j],K[i]] = 1000


for i in range(0,len(gurobi_inputs)):
    a[gurobi_inputs['Stores'][i],gurobi_inputs['SKU'][i]] = gurobi_inputs['Price'][i]*0.05
    i += 1

In [996]:
#Unit cost of open quantity g_hk
for i in range(0,len(K)):
    for j in range(0,len(H)):
        g[H[j],K[i]] = 1000


for i in range(0,len(gurobi_inputs)):
    g[gurobi_inputs['Stores'][i],gurobi_inputs['SKU'][i]] = gurobi_inputs['Price'][i]*0.60
    i += 1

In [997]:
#Price of products p
for i in range(0,len(K)):
    P[K[i]] = 9999

for i in range(0,len(gurobi_inputs)):
    P[gurobi_inputs['SKU'][i]] = gurobi_inputs['Price'][i]
    i += 1

In [998]:
#n_jk Number of product k that can in assortment j
for i in range(0,len(K)):
    for j in range(0,len(J)):
        n[J[j],K[i]] = 0

for i in range(0,len(gurobi_inputs)):
    n[(gurobi_inputs['AssortmentCode'][i], gurobi_inputs['SKU'][i])] = gurobi_inputs['Wmin'][i]
    i += 1

---

#  Model Deployment
## Decision Variables

In [999]:
replenishment = gp.Model('replenishment')

# Number of product k sent from warehouse to store h  vtype=GRB.CONTINUOUS,
x = replenishment.addVars(H, K, vtype=GRB.CONTINUOUS, name="x") 

# Indicator decision variable taking the value 1 if product k sent to store h
y = replenishment.addVars(H, K, vtype=GRB.BINARY, name="y")

# Stock level of product k in store h after replenishment  vtype=GRB.CONTINUOUS. 
I_ending = replenishment.addVars(H, K, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="I_ending") 

# Number of idle stocks of product k in store h #ub=max(0,(I_initial + x) - DU), vtype=GRB.CONTINUOUS,
v = replenishment.addVars(H, K, vtype=GRB.CONTINUOUS, name="v")

# Number of lost sales of product k in store h #ub=max(0,(I_initial + x) - DU), vtype=GRB.CONTINUOUS,
q = replenishment.addVars(H, K, vtype=GRB.CONTINUOUS, name="q")

# continuous variable indicating the amount which false behind the required minimum stock
s = replenishment.addVars(H, K, vtype=GRB.CONTINUOUS, name="s")

# continuous variable number of assortment  j sent to store h
t = replenishment.addVars(H, J,lb = 0, ub = 3, vtype=GRB.INTEGER, name="t") 

# continuous variable number of product k sent to store h outside of an assortment
z = replenishment.addVars(H, K, vtype=GRB.CONTINUOUS, name="z") 

---

## Constraints

In [1000]:
# 2nd v constraint
cons_v = replenishment.addConstrs((v[h,k]
                    >=  I_initial[h,k] + x[h,k] - DU[h,k]
                        for h in H for k in K),
                        name = "cons_v")

In [1001]:
# 3rd v constraint
cons_q = replenishment.addConstrs((q[h,k]
                    >= DL[h,k] - I_initial[h,k] - x[h,k] 
                        for h in H for k in K),
                        name = "cons_q")

In [1002]:
# 4th CC constraint
#cons_CC = replenishment.addConstrs((CC[h,l]
 #                   >= gp.quicksum(e[l,k] * b[h,k] * x[h,k] + e[l,k] * b[h,k] * I_initial[h,k]
  #                      for k in K) for h in H for l in L),
   #                 name = "cons_CC")


cons_CC = replenishment.addConstrs((gp.quicksum(e[l,k] * b[h,k] * x[h,k] + e[l,k] * b[h,k] * I_initial[h,k]
                        for k in K) <= CC[h,l] for h in H for l in L),
                    name = "cons_CC")

In [1003]:
# 6th C constraint
#cons_C = replenishment.addConstrs((C[h]
 #                   >= gp.quicksum(b[h,k] * x[h,k] + b[h,k] * I_initial[h,k]
  #                      for k in K)
   #                     for h in H),
    #                name = "cons_C")


cons_C = replenishment.addConstrs((gp.quicksum(b[h,k] * x[h,k] + b[h,k] * I_initial[h,k]
                        for k in K) <= C[h] for h in H),
                    name = "cons_C")

In [1004]:
# 7th DL_IW constraint
cons_DL_IW = replenishment.addConstrs((min(DL[h,k],IW[k]) * y[h,k]
                    <= b[h,k] * x[h,k] +  b[h,k] * I_initial[h,k]
                        for h in H for k in K),
                        name = "cons_DL_IW")

In [1005]:
# 8th IW constraint
cons_IW = replenishment.addConstrs((gp.quicksum(x[h,k] for h in H)
                                  <= IW[k] for k in K), name = "cons_IW")

In [1006]:
# 9th W2max_DU constraint
cons_W2_max_DU = replenishment.addConstrs((min(W2_max[h,k],DU[h,k]) * y[h,k]
                    >= b[h,k] * x[h,k]
                        for h in H for k in K),
                        name = "W2max_DU")

In [1007]:
# 10th W2_min constraint
cons_W2_min = replenishment.addConstrs((W2_min[h,k] * y[h,k]
                    <= b[h,k] * x[h,k]
                        for h in H for k in K),
                        name = "cons_W2_min")

In [1008]:
# 11th Inventry Balance constraint
cons_balance = replenishment.addConstrs((I_ending[h,k]
                    == b[h,k] * I_initial[h,k] + b[h,k] * x[h,k] - F[h,k]
                        for h in H for k in K),
                        name = "cons_balance")

In [1009]:
# 12th W2_min constraint
cons_balance2 = replenishment.addConstrs((I_ending[h,k] + s[h,k]
                    >= 0
                        for h in H for k in K),
                        name = "cons_balance2")

In [1010]:
# 13th Total number of products shipped
cons_total = replenishment.addConstrs(((gp.quicksum(n[j,k]*t[h,j]
                    for j in J) + z[h,k])  == x[h,k] for h in H for k in K),
                    name = "cons_total")

In [1011]:
# 14 th Minimizing lost sales and idle stock 
obj = (gp.quicksum(a[h,k] * v[h,k] for h in H for k in K) + gp.quicksum(P[k] * (s[h,k] + q[h,k]) for h in H for k in K) + 
      gp.quicksum(g[h,k] * z[h,k] for h in H for k in K))

replenishment.setObjective(obj, GRB.MINIMIZE)

---

In [1012]:
replenishment.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 343962 rows, 304950 columns and 562038 nonzeros
Model fingerprint: 0xb3788bb8
Variable types: 256158 continuous, 48792 integer (42693 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 3e+00]
  RHS range        [5e-01, 1e+05]
Found heuristic solution: objective 519205.37082
Presolve removed 343938 rows and 304929 columns
Presolve time: 0.79s
Presolved: 24 rows, 21 columns, 52 nonzeros
Found heuristic solution: objective 54977.000366
Variable types: 6 continuous, 15 integer (9 binary)

Root relaxation: objective 5.454103e+04, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    54541.

In [1013]:
rows = H.copy()
columns = K.copy()
urunIkmali = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in x.keys():
    if (abs(x[store, product].x) > 1e-6):
        urunIkmali.loc[store, product] = np.round(x[store, product].x, 1)

In [1014]:
rows = H.copy()
columns = K.copy()
y_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in y.keys():
    if (abs(y[store, product].x) > 1e-6):
        y_plan.loc[store, product] = np.round(y[store, product].x, 1)

In [1015]:
rows = H.copy()
columns = K.copy()
v_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in v.keys():
    if (abs(v[store, product].x) > 1e-6):
        v_plan.loc[store, product] = np.round(v[store, product].x, 1)

In [1016]:
rows = H.copy()
columns = K.copy()
I_ending_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in I_ending.keys():
    if (abs(I_ending[store, product].x) > 1e-6):
        I_ending_plan.loc[store, product] = np.round(I_ending[store, product].x, 1)


In [1017]:
rows = H.copy()
columns = K.copy()
q_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in q.keys():
    if (abs(q[store, product].x) > 1e-6):
        q_plan.loc[store, product] = np.round(q[store, product].x, 1)


In [1018]:
rows = H.copy()
columns = J.copy()
t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, AssortmentCode in t.keys():
    if (abs(t[store, AssortmentCode].x) > 1e-6):
        t_plan.loc[store, AssortmentCode] = np.round(t[store, AssortmentCode].x, 1)


In [1019]:
rows = H.copy()
columns = K.copy()
z_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in z.keys():
    if (abs(z[store, product].x) > 1e-6):
        z_plan.loc[store, product] = np.round(z[store, product].x, 1)


In [1020]:
rows = H.copy()
columns = K.copy()
s_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for store, product in s.keys():
    if (abs(s[store, product].x) > 1e-6):
        s_plan.loc[store, product] = np.round(s[store, product].x, 1)